In [396]:
import os
import re
import pandas as pd
import numpy as np

# 1. 東西マージした商品リストの作成

In [397]:
df_product_e = pd.read_csv('data/life_e_all_product_list_20191219.csv',
                          dtype={
                            'item_cd':'str',
                            'clength':'int16',
                            'item_name':'str',
                            'category1_cd':'str',
                            'category1_name':'str',
                            'category2_cd':'str',
                            'category2_name':'str',
                            'category3_cd':'str',
                            'category3_name':'str',
                            'category4_cd':'str',
                            'category4_name':'str'
                          })

df_product_w = pd.read_csv('data/life_w_all_product_list_20191219.csv',
                          dtype={
                            'item_cd':'str',
                            'clength':'int16',
                            'item_name':'str',
                            'category1_cd':'str',
                            'category1_name':'str',
                            'category2_cd':'str',
                            'category2_name':'str',
                            'category3_cd':'str',
                            'category3_name':'str',
                            'category4_cd':'str',
                            'category4_name':'str'
                          })

In [398]:
df_product_e = df_product_e[~df_product_e.duplicated(subset='item_cd', keep='last')]
df_product_w = df_product_w[~df_product_w.duplicated(subset='item_cd', keep='last')]

In [399]:
print('df_product_e:', len(df_product_e))
print('df_product_w:', len(df_product_w))

df_product_e: 128667
df_product_w: 155838


In [400]:
# 縦に結合して全項目重複しているレコードを削除（例：JANが同じでも商品名が違えば残る）

df_product = pd.concat([df_product_e, df_product_w])
df_product.reset_index(drop=True, inplace=True)
df_product = df_product[~df_product.duplicated()]
print('df_product:', len(df_product))

df_product: 249153


In [401]:
# item_cdだけで重複しているレコードを抽出

df_duplicated = df_product[['item_cd', 'item_name']].groupby("item_cd").count()
df_duplicated.reset_index(inplace=True)
df_duplicated.columns = ['item_cd', 'item_count']
df_duplicated = df_duplicated[df_duplicated['item_count'] > 1]
print('df_duplicated:', len(df_duplicated))

df_duplicated: 2806


In [402]:
#項目名を変更してマージ（東のデータには'_e'をつける）
col_list = []
for col in df_product_e.columns:
    col_list.append(col + '_e')
df_product_e.columns = col_list
df_duplicated = pd.merge(df_duplicated, df_product_e, left_on='item_cd', right_on='item_cd_e').drop('item_cd_e', axis=1)

In [403]:
#項目名を変更してマージ（西のデータには'_w'をつける）
col_list = []
for col in df_product_w.columns:
    col_list.append(col + '_w')
df_product_w.columns = col_list
df_duplicated = pd.merge(df_duplicated, df_product_w, left_on='item_cd', right_on='item_cd_w').drop('item_cd_w', axis=1)

In [404]:
# 東西マージ後、item_cdで重複しているものはサブカテゴリなどの微妙な違いだけであることを確認
df_duplicated.to_csv('output/df_duplicated.csv', index=False)
df_duplicated[df_duplicated['item_name_e'] != df_duplicated['item_name_w']]

,item_cd,item_count,clength_e,item_name_e,category1_cd_e,category1_name_e,category2_cd_e,category2_name_e,category3_cd_e,category3_name_e,...,clength_w,item_name_w,category1_cd_w,category1_name_w,category2_cd_w,category2_name_w,category3_cd_w,category3_name_w,category4_cd_w,category4_name_w


In [405]:
# 東西マージしたdf_productに対し、item_cdでユニークにする
df_product = df_product[~df_product.duplicated(subset='item_cd', keep='first')]
print('df_product:', len(df_product))

df_product: 246347


In [406]:
# 上で確認した重複件数と一致するハズ
249153 - 246347

2806

# 2. ライフJAN桁数の調査

In [407]:
df_jan_count = df_product[['clength','item_cd']].groupby('clength').count()
df_jan_count.reset_index(inplace=True)
df_jan_count.columns = ['clength', 'count']

In [408]:
df_jan_count.to_csv('output/df_count.csv', index=False)
df_jan_count

,clength,count
0,1,1
1,2,42
2,3,276
3,4,26
4,6,525
5,7,337
6,8,1832
7,9,3
8,10,391
9,11,139


In [409]:
df_product[df_product['item_cd'].str.contains('^2+.*')]

,item_cd,clength,item_name,category1_cd,category1_name,category2_cd,category2_name,category3_cd,category3_name,category4_cd,category4_name
0,246198000000,12,ベーコンエピ,000092,ベーカリー,909203,ハース・フランス,570300,ハース・フランス,570301,フランスパン
6,246203000000,12,焦がれるチーズフランス,000092,ベーカリー,909203,ハース・フランス,570300,ハース・フランス,570301,フランスパン
14,216007900000,12,豪州産ラム切落２００ｇ▼１便,000011,精肉,901106,バラエティ ミート,731900,バラエティ,731903,ニュージーランドラム
20,232830800000,12,あまに鶏むね肉,000011,精肉,901103,鶏肉,731100,とり正肉,731116,銘柄鶏むね肉
21,296264400000,12,３便◆若どりもも肉１枚,000011,精肉,901103,鶏肉,731100,とり正肉,731101,とり もも肉
...,...,...,...,...,...,...,...,...,...,...,...
284499,2523029043158,13,紳士カジュアルシュー,000552,靴,955201,靴,055201,靴,002302,紳士カジュアルシュー
284500,2523129918158,13,婦人カジュアルシュー,000552,靴,955201,靴,055201,靴,002312,婦人カジュアルシュー
284501,2523059025018,13,紳士ビジネスシューズ,000552,靴,955201,靴,055201,靴,002305,紳士ビジネスシューズ
284503,2523057004060,13,紳士ビジネスシューズ,000552,靴,955201,靴,055201,靴,002305,紳士ビジネスシューズ


In [410]:
df_pivot_table = pd.pivot_table(
    df_product,
    values='item_cd',
    index='category1_name',
    columns='clength',
    aggfunc='count',
    margins=True
).reset_index()

In [411]:
df_pivot_table.to_csv('output/df_pivot_table.csv', index=False)

In [412]:
df_non_stdjan = df_product[
    ((df_product['category1_name'].str.contains('.*酒.*')) & (df_product['item_cd'].str.contains('^2.*')))
    |
    ((df_product['category1_name'].str.contains('.*酒.*')) 
     & (df_product['clength'] != 13)
     & (df_product['clength'] != 8))
]
df_non_stdjan.to_csv('output/df_non_stdjan.csv', index=False)

In [413]:
len(df_non_stdjan)

99

In [414]:
df_product[
    ((df_product['category1_name'].str.contains('.*酒.*')) & (df_product['item_cd'].str.contains('^2.*')))
]

,item_cd,clength,item_name,category1_cd,category1_name,category2_cd,category2_name,category3_cd,category3_name,category4_cd,category4_name
3216,2486603018,10,フルセイル エアストリームＩ,000037,酒類,903708,ビール類,343200,ビール,343202,輸入
16274,2486619017,10,フルセイル セッションイージ,000037,酒類,903708,ビール類,343200,ビール,343202,輸入
30109,2486622018,10,フルセイル アトマイザーイン,000037,酒類,903708,ビール類,343200,ビール,343202,輸入
132366,25,2,スーパードライ３５０ｍｌＣＳ,000037,酒類,903708,ビール類,343200,ビール,343201,国産


# 3. サントリーマスタの調査

In [415]:
df_suntory = pd.read_csv('data/suntory_product_master_20191220.csv',
                        dtype={
                            'jancd':'str', 'makercd':'str', 'itemnm':'str', 'itemkn':'str', 'tanka':'int32','itemkikaku':'float32',
                            'itemtani':'str', 'irisu':'int32', 'width':'int32','height':'int32', 'depth':'int32', 
                            'jcat1cd':'str', 'jcat2cd':'str', 'jcat3cd':'str','jcat4cd':'str', 'jcatnm':'str',
                            'scat1cd':'str', 'scat2cd':'str', 'scat3cd':'str','scat4cd':'str',
                            'scat1nm':'str', 'scat2nm':'str', 'scat3nm':'str','scat4nm':'str',
                            'syubaiflg':'str', 'hatdate':'str', 'instime':'str', 'updtime':'str','delete_flg':'str'}
                        )

In [416]:
df_suntory.dtypes

jancd          object
makercd        object
itemnm         object
itemkn         object
tanka           int32
itemkikaku    float32
itemtani       object
irisu           int32
width           int32
height          int32
depth           int32
jcat1cd        object
jcat2cd        object
jcat3cd        object
jcat4cd        object
jcatnm         object
scat1cd        object
scat2cd        object
scat3cd        object
scat4cd        object
scat1nm        object
scat2nm        object
scat3nm        object
scat4nm        object
syubaiflg      object
hatdate        object
instime        object
updtime        object
delete_flg     object
dtype: object

In [417]:
df_suntory.head(3)

,jancd,makercd,itemnm,itemkn,tanka,itemkikaku,itemtani,irisu,width,height,...,scat4cd,scat1nm,scat2nm,scat3nm,scat4nm,syubaiflg,hatdate,instime,updtime,delete_flg
0,00000018,0001777,ｻﾝﾄﾘｰ スーパーホップス6缶パック <ケースカット> 500ml,Sｽｰﾊﾟｰﾎｯﾌﾟｽ500ｶﾝﾊｺ,0,500.0,ml,0,270,170,...,001,飲料･酒類,ﾋﾞｰﾙ･発泡酒,新ｼﾞｬﾝﾙ,新ｼﾞｬﾝﾙ,1,NaN,2009-06-05 00:00:00,2019-05-31 00:00:00,0
1,00000019,999999999,酒,Sﾑｷﾞﾉｾﾞｲﾀｸ5006ｶﾝｹｰｽ,0,3000.0,ml,0,270,170,...,001,飲料･酒類,ﾋﾞｰﾙ･発泡酒,新ｼﾞｬﾝﾙ,新ｼﾞｬﾝﾙ,1,1900-01-01,2009-06-05 00:00:00,2019-05-31 00:00:00,0
2,00000050,0001777,ｻﾝﾄﾘｰ スーパーホップスマグナムドライ 350ml 6缶ケースカット,Sｽｰﾊﾟｰﾎｯﾌﾟｽ(ﾏｸﾞﾅﾑﾄﾞﾗｲ)350ｹｰｽｶｯ,0,2100.0,ml,0,270,130,...,001,飲料･酒類,ﾋﾞｰﾙ･発泡酒,新ｼﾞｬﾝﾙ,新ｼﾞｬﾝﾙ,1,NaN,2009-06-05 00:00:00,2019-05-31 00:00:00,0


In [510]:
# 45もしくは49から始まる数字だけのJAN（これは正規JANとみなす）を除外するとどうなるか確認する
df_suntory[['jancd','itemnm']][~(df_suntory['jancd'].str.match('^(45|49)[0-9]+$'))].to_csv('output/non_jp.csv', index=False)

In [514]:
# サントリーマスタにJAN桁数の列を追加する
df_suntory['clength'] = df_suntory['jancd'].apply(lambda x: len(str(x)))

In [516]:
df_suntory[['jancd', 'clength']].groupby('clength').count()

,jancd
clength,
2,4
3,19
4,12
5,17
6,44
7,2876
8,627
9,11
10,2


In [522]:
df_suntory[['jancd', 'itemnm']][df_suntory['clength'] == 8].to_csv('output/jan8.csv', index = False)